In [ ]:

import re
import random
from collections import Counter, defaultdict
import math

# ----------------------------
# 1) Offline cleaning utilities
# ----------------------------
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = text.split()
    return tokens

simple_stopwords = set("""
a an the and or if in on at to for of this that is was were be been it with as by from into up out 
do does did doing have has had not so such these those he she they them we you i but which will would
should can could may might also about over under between during per each eachother
""".split())

def remove_stopwords(tokens):
    return [t for t in tokens if t not in simple_stopwords]

def perform_lemmatization(tokens):
    lemmas = []
    for t in tokens:
        if t.endswith("ing") and len(t) > 4:
            lemmas.append(t[:-3])
        elif t.endswith("ed") and len(t) > 3:
            lemmas.append(t[:-2])
        elif t.endswith("s") and len(t) > 3:
            lemmas.append(t[:-1])
        else:
            lemmas.append(t)
    return lemmas

def clean_text(text):
    tokens = preprocess_text(text)
    tokens = remove_stopwords(tokens)
    tokens = perform_lemmatization(tokens)
    return tokens

# ----------------------------
# 2) Load file (change filename if needed)
# ----------------------------
FILENAME = "0000320187-25-000053.txt"  # change if your file has another name

with open(FILENAME, "r", encoding="utf-8") as f:
    raw = f.read()

tokens = clean_text(raw)

# Print the same basic diagnostics you showed earlier:
print(f"Total tokens after cleaning: {len(tokens)}")
print("Sample tokens:", tokens[:40])

# Frequency and ASCII word cloud (top 30)
freq = Counter(tokens)
most_common_30 = freq.most_common(30)

print("\nTop 30 most common words:")
for w, c in most_common_30:
    print(f"{w}: {c}")

print("\nASCII Word Cloud:")
max_width = 50
for word, count in most_common_30:
    bar = "#" * min(count, max_width)
    print(f"{word:15} {bar}")

# ----------------------------
# 3) Build pseudo-documents
# If you have true documents, replace this step and provide a list of token lists.
# Here we chunk the token list into multiple small documents to run LDA.
# ----------------------------
TOTAL_TOKENS = len(tokens)
# Choose number of pseudo-documents (roughly one doc per 150-300 tokens)
DOC_CHUNK_SIZE = 200
if TOTAL_TOKENS <= DOC_CHUNK_SIZE:
    documents = [tokens]   # single short doc
else:
    documents = []
    for i in range(0, TOTAL_TOKENS, DOC_CHUNK_SIZE):
        documents.append(tokens[i : i + DOC_CHUNK_SIZE])

# Optionally: make number of documents at least 5 for stable topics
if len(documents) < 5 and TOTAL_TOKENS > 0:
    # re-chunk with smaller chunk size
    DOC_CHUNK_SIZE = max(20, TOTAL_TOKENS // 8)
    documents = [tokens[i : i + DOC_CHUNK_SIZE] for i in range(0, TOTAL_TOKENS, DOC_CHUNK_SIZE)]

num_docs = len(documents)
print(f"\nBuilt {num_docs} pseudo-documents (chunk size ~{DOC_CHUNK_SIZE}).")

# ----------------------------
# 4) Prepare vocabulary and ids
# ----------------------------
vocab = sorted(list(set(tokens)))
V = len(vocab)
word2id = {w: i for i, w in enumerate(vocab)}
id2word = {i: w for w, i in word2id.items()}

# Convert documents to id-lists
docs_wids = [[word2id[w] for w in doc] for doc in documents]

# ----------------------------
# 5) Simple collapsed Gibbs sampler for LDA
# ----------------------------
NUM_TOPICS = 6         # change number of topics
ALPHA = 0.1            # doc-topic prior
BETA = 0.01            # topic-word prior
ITERATIONS = 400       # sampling iterations (increase for better convergence)
TOP_N_WORDS = 15       # words to print per topic

random.seed(0)

D = len(docs_wids)
K = NUM_TOPICS

# counts
doc_topic_counts = [defaultdict(int) for _ in range(D)]   # ndk
topic_word_counts = [defaultdict(int) for _ in range(K)]  # nkw
topic_counts = [0 for _ in range(K)]                      # nk
doc_lengths = [len(doc) for doc in docs_wids]

# initial topic assignments (random)
topic_assignments = []
for d, doc in enumerate(docs_wids):
    current_doc_topics = []
    for w in doc:
        z = random.randrange(K)
        current_doc_topics.append(z)
        doc_topic_counts[d][z] += 1
        topic_word_counts[z][w] += 1
        topic_counts[z] += 1
    topic_assignments.append(current_doc_topics)

# Precompute constants
V_beta = V * BETA

def sample_topic(d, w, current_z):
    # remove current assignment
    doc_topic_counts[d][current_z] -= 1
    if doc_topic_counts[d][current_z] == 0:
        del doc_topic_counts[d][current_z]
    topic_word_counts[current_z][w] -= 1
    if topic_word_counts[current_z][w] == 0:
        del topic_word_counts[current_z][w]
    topic_counts[current_z] -= 1

    # compute full conditional for each topic
    probs = []
    for k in range(K):
        left = (doc_topic_counts[d].get(k, 0) + ALPHA) / (doc_lengths[d] - 1 + K * ALPHA)
        right = (topic_word_counts[k].get(w, 0) + BETA) / (topic_counts[k] + V_beta)
        probs.append(left * right)

    # normalize
    s = sum(probs)
    if s == 0:
        probs = [1.0 / K] * K
    else:
        probs = [p / s for p in probs]

    # draw new topic
    r = random.random()
    cum = 0.0
    for k, p in enumerate(probs):
        cum += p
        if r <= cum:
            new_z = k
            break

    # add back new assignment
    doc_topic_counts[d][new_z] += 1
    topic_word_counts[new_z][w] += 1
    topic_counts[new_z] += 1

    return new_z

# Gibbs sampling iterations
print("\nRunning Gibbs sampling...")
for it in range(1, ITERATIONS + 1):
    for d, doc in enumerate(docs_wids):
        for i, w in enumerate(doc):
            current_z = topic_assignments[d][i]
            new_z = sample_topic(d, w, current_z)
            topic_assignments[d][i] = new_z

    if it % 100 == 0 or it == 1:
        print(f"Iteration {it}/{ITERATIONS}")

print("Gibbs sampling finished.")

# ----------------------------
# 6) Compute topic-word distributions (phi) and print topics
# ----------------------------
# phi[k][w] = (nkw + beta) / (nk + V*beta)
phi = []
for k in range(K):
    phi_k = {}
    denom = topic_counts[k] + V_beta
    for w_id in topic_word_counts[k]:
        phi_k[w_id] = (topic_word_counts[k].get(w_id, 0) + BETA) / denom
    # include zeros for unseen words if needed (not necessary for top words)
    phi.append(phi_k)

# Print topics in requested format (topic index + top words with weights)
print("\nTopics (top words with weights):")
for k in range(K):
    # get top words for topic k by phi
    # If topic has no words (rare), skip
    if not phi[k]:
        print(f"\nTopic {k+1}: (empty)")
        continue

    # create list of (word, prob)
    word_probs = [(id2word[w_id], prob) for w_id, prob in phi[k].items()]
    word_probs.sort(key=lambda x: x[1], reverse=True)
    top = word_probs[:TOP_N_WORDS]

    print(f"\nTopic {k+1}:")
    for prob_word in top:
        prob = prob_word[1]
        word = prob_word[0]
        # print weight with 3 decimals like "0.123 word"
        print(f"{prob:.3f} {word}")

# ----------------------------
# 7) Optionally, show topic-word counts for inspection
# ----------------------------
# Example: most probable words across topics (concise)
print("\nTop words per topic (concise):")
for k in range(K):
    if not phi[k]:
        print(f"Topic {k+1}: (empty)")
        continue
    top_words = sorted(phi[k].items(), key=lambda x: x[1], reverse=True)[:8]
    print("Topic %d:" % (k+1), ", ".join([f"{id2word[w_id]}" for w_id, _ in top_words]))


Total tokens after cleaning: 1870
Sample tokens: ['sec', 'document', 'txt', 'sec', 'header', 'hdr', 'sgml', 'acceptance', 'datetime', 'accession', 'number', 'conform', 'submission', 'type', 'k', 'public', 'document', 'count', 'conform', 'period', 'report', 'item', 'information', 'other', 'event', 'fil', 'date', 'date', 'change', 'filer', 'company', 'data', 'company', 'conform', 'name', 'nike', 'inc', 'central', 'index', 'key']

Top 30 most common words:
type: 92
dei: 56
name: 52
document: 47
definition: 36
reference: 33
entity: 32
number: 29
period: 29
exchange: 27
act: 26
data: 24
text: 24
prefix: 24
xbrl: 23
x: 23
detail: 23
namespace: 23
balance: 23
na: 23
duration: 23
sec: 21
addres: 20
false: 20
report: 19
b: 19
fil: 18
k: 17
company: 15
nke: 15

ASCII Word Cloud:
type            ##################################################
dei             ##################################################
name            ##################################################
document        ###